<a href="https://colab.research.google.com/github/pragnavi/Machine-Learning/blob/master/The_SVM_vs_Logistic_Regression_Showdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The SVM vs. Logistic Regression Showdown

In this lab, you will practice working with non-linear kernels combined with logistic regression and SVM classifiers. The goal is to compare these commonly used techniques. Which comes out on top in terms of accuracy? Runtime? Is there much of a difference at all? Is the dominance of the SVM classifier in machine learning pedagogy justified?

## Loading the Data

First, we load all the packages we'll need.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import sklearn.metrics.pairwise as pairwise
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time
from datetime import timedelta
from sklearn.metrics import accuracy_score

Again we download the data from the Tensorflow package, which you will need to install.  You can get the data from other sources as well.

In the Tensorflow dataset, the training and test data are represented as arrays:

     Xtr.shape = 60000 x 28 x 28
     Xts.shape = 10000 x 28 x 28
     
The test data consists of `60000` images of size `28 x 28` pixels; the test data consists of `10000` images.

In [ ]:
import tensorflow as tf

(Xtr_raw,ytr),(Xts_raw,yts) = tf.keras.datasets.mnist.load_data()

print('Xtr shape: %s' % str(Xtr_raw.shape))
print('Xts shape: %s' % str(Xts_raw.shape))

ntr = Xtr_raw.shape[0]
nts = Xts_raw.shape[0]
nrow = Xtr_raw.shape[1]
ncol = Xtr_raw.shape[2]

11490434/11490434 [==============================] - 0s 0us/step
Xtr shape: (60000, 28, 28)
Xts shape: (10000, 28, 28)


Each pixel value is from `[0,255]`.  For this lab, it will be convenient to recale the value to -1 to 1 and reshape it as a `ntr x npix` and `nts x npix`.

In [ ]:
npix = nrow*ncol
Xtr = Xtr_raw.reshape((ntr,npix))
print(Xtr[1,:])
Xtr = (Xtr/255 - .5)
print(Xtr[1,:])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  51 159 253 159  50   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252
 253 122   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 163 252 252 252 253 252 252  96 189 253 167   

In [ ]:
npix = nrow*ncol
Xtr = (Xtr_raw/255 - 0.5)
Xtr = Xtr.reshape((ntr,npix))

Xts = (Xts_raw/255 - 0.5)
Xts = Xts.reshape((nts,npix))

In [ ]:
print(Xtr)

[[-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 ...
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]]


For this lab we're only going to use a fraction of the MNIST data -- otherwise our models will take too much time and memory to run. Using only part of the training data will of course lead to worse results. Given enough computational resources and time, we would ideally be running on the full data set. The follow code creates a new test and train set, with 10000 examples for train and 5000 for test.

In [ ]:
ntr1 = 10000
nts1 = 5000
Iperm = np.random.permutation(ntr1)
Xtr1 = Xtr[Iperm[:ntr1],:]
ytr1 = ytr[Iperm[:ntr1]]
Iperm = np.random.permutation(nts1)
Xts1 = Xts[Iperm[:nts1],:]
yts1 = yts[Iperm[:nts1]]

## Problem set up and establishing a baseline

To simplify the problem (and speed things up) we're also going to restrict to binary classification. In particular, let's try to design classifier a that separates the 8's from all other digits.

Create binary 0/1 label vectors `ytr8` and `yts8` which are 1 wherever `ytr1` and `yts1` equal 8, and 0 everywhere else.

In [ ]:
# TODO
ytr8 = [int(ytr == 8) for ytr in ytr1]
yts8 = [int(yts == 8) for yts in yts1]

Most of the digits in the test dataset aren't equal to 8. So if we simply guess 0 for every image in `Xts`, we might expect to get classification accuracy around 90%. Our goal should be to significantly beat this **baseline**.

Formally, write a few lines of code to check what test error would be achieved by the all zeros classifier.

In [ ]:
# TODO
yts0 = [0 for yts in yts1]
acc = accuracy_score(yts8,yts0)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.902200


As a second baseline, let's see how we do with standard (non-kernel) logistic regression. As in the MNIST demo, you can use `scikit-learn`'s built in function `linear_model.LogisticRegression` to fit the model and compute the accuracy. Use no regularization and the `lbfgs` solver. You should acheive an improvement to around 93-95%.

In [ ]:
# TODO
logreg = linear_model.LogisticRegression(verbose=10, solver='lbfgs', multi_class='multinomial',max_iter=1000)
logist = logreg.fit(Xtr1,ytr8)
ypred = logist.predict(Xts1)
acc = accuracy_score(yts8,ypred)
print('Logistic Regression Accuaracy = {0:f}'.format(acc))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Logistic Regression Accuaracy = 0.942000


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min finished


## Kernel Logistic Regression

To improve on this baseline performance, let's try using the logistic regression classifier with a *non-linear* kernel. Recall from class that any non-linear kernel similarity function $k(\vec{w},\vec{z})$ is equal to $\phi(\vec{w})^T\phi(\vec{z})$ for some feature transformation $\phi$. However, we typically do not need to compute this feature tranformation explicitly: instead we can work directly with the kernel gram matrix $K \in \mathbb{R}^{n\times n}$. Recall that $K_{i,j} = k(\vec{x}_i,\vec{x}_j)$ where $\vec{x}_i$ is the $i^\text{th}$ training data point.

For this lab we will be using the radial basis function kernel. For a given scaling factor $\gamma$ this kernel is defined as:
$$
k(\vec{w},\vec{z}) = e^{-\gamma\|\vec{w}-\vec{z}\|_2^2}
$$

In [ ]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

Construct the kernel matrix `K1` for `Xtr1` with `gamma = .05`.

In [ ]:
# TODO,
gamma = .05
temp = np.zeros((Xtr1.shape[0],Xtr1.shape[0]))
K1 = np.array(temp)
for i in range(Xtr1.shape[0]):
  for j in range(i):
    K1[i,j] = rbf_kernel(Xtr1[i,:],Xtr1[j,:],gamma)
    K1[j,i] = K1[i,j]
K1 = np.identity(Xtr1.shape[0])+K1
print(K1)

[[1.00000000e+00 4.01887410e-02 3.43326152e-04 ... 4.87743171e-03
  2.77660389e-03 6.15093077e-03]
 [4.01887410e-02 1.00000000e+00 9.58916778e-04 ... 4.18840690e-03
  1.23075858e-03 2.52484919e-02]
 [3.43326152e-04 9.58916778e-04 1.00000000e+00 ... 6.40943609e-05
  2.71354280e-03 4.56902577e-04]
 ...
 [4.87743171e-03 4.18840690e-03 6.40943609e-05 ... 1.00000000e+00
  1.37587324e-03 1.06864310e-02]
 [2.77660389e-03 1.23075858e-03 2.71354280e-03 ... 1.37587324e-03
  1.00000000e+00 9.12243144e-04]
 [6.15093077e-03 2.52484919e-02 4.56902577e-04 ... 1.06864310e-02
  9.12243144e-04 1.00000000e+00]]


If you used a for loop (which is fine) your code might take several minutes to run! Part of the issue is that Python won't know to properly parallize your for loop. For this reason, when constructing kernel matrices it is often faster to us a built-in, carefully optimized function with explicit parallelization. Scikit learn provides such a function through their `metrics` library.

Referring to the documentation here
https://scikit-learn.org/stable/modules/metrics.html#metrics, use this built in function to recreate the same kernel matrix you did above. Store the result at `K`.

In [ ]:
# TODO
K = pairwise_kernels(Xtr1, metric = 'rbf',gamma = .05)
print(K)

[[1.00000000e+00 4.01887410e-02 3.43326152e-04 ... 4.87743171e-03
  2.77660389e-03 6.15093077e-03]
 [4.01887410e-02 1.00000000e+00 9.58916778e-04 ... 4.18840690e-03
  1.23075858e-03 2.52484919e-02]
 [3.43326152e-04 9.58916778e-04 1.00000000e+00 ... 6.40943609e-05
  2.71354280e-03 4.56902577e-04]
 ...
 [4.87743171e-03 4.18840690e-03 6.40943609e-05 ... 1.00000000e+00
  1.37587324e-03 1.06864310e-02]
 [2.77660389e-03 1.23075858e-03 2.71354280e-03 ... 1.37587324e-03
  1.00000000e+00 9.12243144e-04]
 [6.15093077e-03 2.52484919e-02 4.56902577e-04 ... 1.06864310e-02
  9.12243144e-04 1.00000000e+00]]


Check that you used the function correctly by writing code to confirm that `K = K1`, or at least that the two are equal up to very small differences (which could arise due to numerical precision issues). Try to do this **without a for loop** for full credit. You will get partial credit if you use a for loop.

In [ ]:
# TODO
rtol = 1e-05
atol = 1e-08
print("Are the two arrays are equal within the tolerance: ", np.allclose(K1, K, rtol, atol))

Are the two arrays are equal within the tolerance:  True


When using a non-linear kernel, it is important to check that you have chosen reasonable parameters (in our case the only parameter is `gamma`). We typically do not want $k(\vec{x}_i,\vec{x}_j)$ to be either negligably small, or very large for all $i\neq j$ in our data set, or we won't be able to learn anything. For the RBF kernel this means that, for any $\vec{x}_i$ we don't want $k(\vec{x}_i,\vec{x}_j)$ very close to 1 (e.g. .9999) for all $j$, or very close to $0$ (e.g. 1e-8) for all $j$.

Let's just check that we're in good shape for the first data vector $\vec{x}_0$. Do so by printing out the 10 largest and 10 smallest values of $k(\vec{x}_0,\vec{x}_j)$ for $j\neq 0$. Note that we always have $k(\vec{x}_0,\vec{x}_0) = 1$ for the RBF kernel.

In [ ]:
# TODO
temp = np.sort(K[0,:])
max = temp[-11:]
max = max[:10]
min = temp[:10]
print('Maximum similarities: \n', max)
print('Minimum similarities: \n', min)

Maximum similarities: 
 [0.67226966 0.6724501  0.67378028 0.6799054  0.75903644 0.78187413
 0.84119162 0.8424759  0.84727423 0.89295529]
Minimum similarities: 
 [2.45843754e-05 3.03996697e-05 3.14632697e-05 3.47787083e-05
 3.59048209e-05 4.92451471e-05 5.07939954e-05 5.12365171e-05
 5.15547700e-05 5.33418319e-05]


### Implementation
Maybe surprisingly Scikit learn does not have an implementation for kernel logistic regression. So we have to implement our own!

Write a function function `log_fit` that minimizes the $\ell_2$-regularized logisitic regression loss:
$$
L(\boldsymbol{\alpha}) =\sum_{i=1}^n (1-y_i)(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\vec{\alpha}) - \log(h(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\boldsymbol{\alpha}) + \lambda \|\phi(\mathbf{X})^T\boldsymbol{\alpha}\|_2^2.
$$
As input it takes an $n\times n$ kernel matrix $K$ for the training data, an $n$ length vector `y` of binary class labels, and regularization parameter `lamb`.

To implement your function you can either use your own implmentation of gradient descent or used a built in minimizer from `scipy.optimize.minimize`. I recommend the later approach: as we saw in the last lab, gradient descent can converge slowly for this objective.

In [ ]:
# TODO
def Leval_reg_linear_kernel(alpha,y,lamb,K):
  h = scipy.special.expit(K@alpha)
  Lr = np.sum((1-y)*(K@alpha) - np.log(h)) + lamb*(np.transpose(alpha))@K@alpha
  Lgrad = K@(h-y) + 2*lamb*(K@alpha)
  return Lr, Lgrad

def log_fit(K,y,lamb,nit=1000):
  alpha0 = np.zeros(Xtr1.shape[0])
  f = lambda alpha: Leval_reg_linear_kernel(alpha,y,lamb, K)[0]
  g = lambda alpha: Leval_reg_linear_kernel(alpha,y,lamb, K)[1]
  res = scipy.optimize.minimize(f, alpha0, args=(), method='L-BFGS-B',jac=g, options={"maxiter":nit})
  alpha = res.x
  return alpha

#    Function which minizes the regularized logistic regression loss for a given kernel matrix K and target vector y
#    Return the optimal parameters alpha of the logisitic regression model.

Use the `log_fit` function defined above to find parameters `alpha` for the kernel logistic regression model using `lamb = 0` and `K` as constructed above (with `gamma = .05`).

In [ ]:
# TODO
lamb=0
ytr8 = np.array(ytr8)
alpha = log_fit(K,ytr8,lamb)

In [ ]:
print(alpha, yts1.shape)

[  2.56982859   2.49111889  -4.19803021 ... -22.29355185  -8.37963983
   0.48648271] (5000,)


Suppose we have a test dataset with $m$ examples $\vec{w}_1,\ldots, \vec{w}_m$. Once we obtain a coefficient vector $\alpha$, making predictions for any $\vec{w}_j$ in the test set requires computing:
$$
{y}_{j} = \sum_{i=1}^n \alpha_i \cdot k(\vec{w}_{j}, \vec{x}_i).
$$
where $\vec{x}_1, \ldots \vec{x}_n$ are our training data vectors. We classify $\vec{w}_{j}$ in class 0 if ${y}_{j} \leq 0$ and in class 1 if ${y}_{j} > 0$.

This computation can be rewritten in matrix form as follows:
$$
\vec{y}_{test} = K_{test}\vec{\alpha},
$$
where $\vec{y}_{text}$ is an $m$ length vector and $K_{test}$ is a $m\times n$ matrix whose $(j,i)$ entry is equal to $k(\vec{w}_{j}, \vec{x}_i)$. We classify $\vec{w}_{j}$ in class 0 if $\vec{y}_{test}[j] \leq 0$ and in class 1 if $\vec{y}_{test}[j] > 0$.


Use the `pairwise_kernels` function to construct $K_{test}$. Then make predictions for the test set and evaluate the accuracy of our kernel logistic regression classifier. You should see a pretty substantial lift in accuracy to around $97\%$

In [ ]:
# TODO

Ktest = pairwise_kernels(Xts1, Xtr1, metric = 'rbf',gamma = .05)

In [ ]:
# TODO
def test_ac(Kt,alph,y):
  yhat = Kt@alph #(vector containing predicted 0,1 labels)
  yhat = [int(yt > 0) for yt in yhat]
  yhat = np.array(yhat)
  acc = np.mean(yhat == y)
  error = np.mean(yhat != y)
  return acc,error
acc,error = test_ac(Ktest,alpha,yts8)
print("Test accuracy = %f" % acc)

Test accuracy = 0.979800


## Kernel Support Vector Machine

The goal of this lab is to compare Kernel Logistic Regression to Kernel Support Vector machines. Following `demo_mnist_svm.ipynb` create and train an SVM classifier on `Xtr1` and `ytr8` using an RBF kernel with `gamma = .05` (the same value we used for logistic regression aboe). Use margin parameter `C = 10`.

In [ ]:
# TODO
svcrbf = svm.SVC(probability=False,  kernel="rbf", C=10, gamma=.05 ,verbose=10)
svcrbf.fit(Xtr1,ytr8)

[LibSVM]

SVC(C=10, gamma=0.05, verbose=10)

Calculate and print the accuracy of the SVM classifier. You should obtain a similar result as for logistic regression: something close to $97\%$ accuracy.

In [ ]:
# TODO

ysvm = svcrbf.predict(Xts1) #(vector containing predicted 0,1 labels)
acc = np.mean(ysvm == yts8)
print("Test accuracy = %f" % acc)

Test accuracy = 0.980200


## The Showdown

Both SVM classifiers and kernel logisitic regression require tuning parameters to obtain the best possible result. In our setting we will stick with an RBF kernel (although this could be tuned). So we only consider tuning the kernel width parameter `gamma`, as well as the regularization parameter `lamb` for logistic regression, and the margin parameter `C` for SVM. We will choose parameters using for-loops and train-test cross validation.

Train a logistic regression classifier with **all combinations** of the parameters included below in vectors `gamma` and `lamb`. For each setting of parameters, compute and print:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

For computing runtime you might want to use the `time()` function from the `time` library, which we already imported ealier.

In [ ]:
gamma = [.1, .05,.02,.01,.005]
lamb = [0,1e-6,1e-4,1e-2]
minerror = float("inf")
opt_gamma = gamma[0]
opt_lamb = lamb[0]
tot_time = 0

# TODO
for gam in gamma:
  begin = time.time()
  K = pairwise_kernels(Xtr1, metric = 'rbf',gamma = gam)
  Ktest = pairwise_kernels(Xts1, Xtr1, metric = 'rbf',gamma = gam)
  finish = time.time()
  timedif = finish-begin
  for lam in lamb:
    start = time.time()
    alpha = log_fit(K,ytr8,lam)
    acc,error = test_ac(Ktest,alpha,yts8)
    if(error<minerror):
      minerror = error
      opt_gamma = gam
      opt_lamb = lam
      opt_acc = acc
    print("Test accuracy for lamb = %f', gamma = '%f' is '%f'" % (lam,gam,acc))
    print("Test error for lamb = %f', gamma = '%f' is '%f'" % (lam,gam,error))
    end = time.time()
    duration = end-start
    tot_time += duration
    print("Time taken for this iteration: "+str(timedelta(seconds = int(round(duration)))))
  tot_time += timedif

print("At gamma = %f and lamb = %f with accuracy = %f minimum test error = %f is obtained" %(opt_gamma,opt_lamb, opt_acc,minerror))
print("Total time taken: "+str(timedelta(seconds = int(round(tot_time)))))

Test accuracy for lamb = 0.000000', gamma = '0.100000' is '0.963200'
Test error for lamb = 0.000000', gamma = '0.100000' is '0.036800'
Time taken for this iteration: 0:00:20
Test accuracy for lamb = 0.000001', gamma = '0.100000' is '0.965800'
Test error for lamb = 0.000001', gamma = '0.100000' is '0.034200'
Time taken for this iteration: 0:00:53
Test accuracy for lamb = 0.000100', gamma = '0.100000' is '0.965200'
Test error for lamb = 0.000100', gamma = '0.100000' is '0.034800'
Time taken for this iteration: 0:01:17
Test accuracy for lamb = 0.010000', gamma = '0.100000' is '0.962800'
Test error for lamb = 0.010000', gamma = '0.100000' is '0.037200'
Time taken for this iteration: 0:01:20
Test accuracy for lamb = 0.000000', gamma = '0.050000' is '0.979800'
Test error for lamb = 0.000000', gamma = '0.050000' is '0.020200'
Time taken for this iteration: 0:00:37
Test accuracy for lamb = 0.000001', gamma = '0.050000' is '0.979800'
Test error for lamb = 0.000001', gamma = '0.050000' is '0.020

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Test accuracy for lamb = 0.000000', gamma = '0.020000' is '0.981400'
Test error for lamb = 0.000000', gamma = '0.020000' is '0.018600'
Time taken for this iteration: 0:03:35
Test accuracy for lamb = 0.000001', gamma = '0.020000' is '0.984800'
Test error for lamb = 0.000001', gamma = '0.020000' is '0.015200'
Time taken for this iteration: 0:12:50
Test accuracy for lamb = 0.000100', gamma = '0.020000' is '0.984200'
Test error for lamb = 0.000100', gamma = '0.020000' is '0.015800'
Time taken for this iteration: 0:12:51
Test accuracy for lamb = 0.010000', gamma = '0.020000' is '0.982200'
Test error for lamb = 0.010000', gamma = '0.020000' is '0.017800'
Time taken for this iteration: 0:12:39


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Test accuracy for lamb = 0.000000', gamma = '0.010000' is '0.977800'
Test error for lamb = 0.000000', gamma = '0.010000' is '0.022200'
Time taken for this iteration: 0:12:47
Test accuracy for lamb = 0.000001', gamma = '0.010000' is '0.981600'
Test error for lamb = 0.000001', gamma = '0.010000' is '0.018400'
Time taken for this iteration: 0:13:20
Test accuracy for lamb = 0.000100', gamma = '0.010000' is '0.982800'
Test error for lamb = 0.000100', gamma = '0.010000' is '0.017200'
Time taken for this iteration: 0:12:41
Test accuracy for lamb = 0.010000', gamma = '0.010000' is '0.980400'
Test error for lamb = 0.010000', gamma = '0.010000' is '0.019600'
Time taken for this iteration: 0:12:46
Test accuracy for lamb = 0.000000', gamma = '0.005000' is '0.976800'
Test error for lamb = 0.000000', gamma = '0.005000' is '0.023200'
Time taken for this iteration: 0:13:17
Test accuracy for lamb = 0.000001', gamma = '0.005000' is '0.976000'
Test error for lamb = 0.000001', gamma = '0.005000' is '0.024

At gamma = 0.020000 and lamb = 0.000001 with accuracy = 0.984800, minimum test error = 0.015200 is obtained. The kernel logistic regression is only slightly sensitive to changes in gamma and lambda. The test error is only slightly varying from one combination to the other combination of gamma and lambda.

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Was the kernel logistic regression classifier more sensitive to changes in `gamma` or `lamb`? Discuss in 1-3 short sentences below.

Now let's do the same thing for the kernel Support Vector Classifier. Train an SVM classifier with **all combinations** of the parameters included below in vectors `gamma` and `C`. For each setting of parameters, compute:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

In [ ]:
gamma = [.1, .05,.02,.01,.005]
C = [.01,.1,1,10]
minerror = float("inf")
opt_gamma = gamma[0]
opt_c = C[0]
tot_time = 0
# TODO
for gam in gamma:
  for c in C:
    start = time.time()
    svcrbf = svm.SVC(probability=False,  kernel="rbf", C=10, gamma=.05 ,verbose=10)
    svcrbf.fit(Xtr1,ytr8)
    ysvm = svcrbf.predict(Xts1) #(vector containing predicted 0,1 labels)
    acc = np.mean(ysvm == yts8)
    error = np.mean(ysvm != yts8)
    if(error<minerror):
      minerror = error
      opt_gamma = gam
      opt_c = c
      opt_acc = acc
    print("Test error for C = %f', gamma = '%f' is '%f'" % (c,gam,error))
    end = time.time()
    duration = end-start
    tot_time += duration
    print("Time taken for this iteration: "+str(timedelta(seconds = int(round(duration)))))
# TODO
print("At gamma = %f and C = %f with accuracy = %f minimum test error = %f is obtained" %(opt_gamma,opt_c, opt_acc,minerror))
print("Total time taken: "+str(timedelta(seconds = int(round(tot_time)))))

[LibSVM]Test error for C = 0.010000', gamma = '0.100000' is '0.019800'
Time taken for this iteration: 0:00:40
[LibSVM]Test error for C = 0.100000', gamma = '0.100000' is '0.019800'
Time taken for this iteration: 0:00:40
[LibSVM]Test error for C = 1.000000', gamma = '0.100000' is '0.019800'
Time taken for this iteration: 0:00:39
[LibSVM]Test error for C = 10.000000', gamma = '0.100000' is '0.019800'
Time taken for this iteration: 0:00:39
[LibSVM]Test error for C = 0.010000', gamma = '0.050000' is '0.019800'
Time taken for this iteration: 0:00:39
[LibSVM]Test error for C = 0.100000', gamma = '0.050000' is '0.019800'
Time taken for this iteration: 0:00:39
[LibSVM]Test error for C = 1.000000', gamma = '0.050000' is '0.019800'
Time taken for this iteration: 0:00:39
[LibSVM]Test error for C = 10.000000', gamma = '0.050000' is '0.019800'
Time taken for this iteration: 0:00:39
[LibSVM]Test error for C = 0.010000', gamma = '0.020000' is '0.019800'
Time taken for this iteration: 0:00:40
[LibSVM]

At gamma = 0.100000 and C = 0.010000 with accuracy = 0.980200, minimum test error = 0.019800 is obtained. In terms of accuracy, both Kernel logistic regression and SVM performed equally well. Their best accuracies are almost the same in both the cases. But SVM performs a lot better than logisitic regression when runtime is considered.

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Which performed better in terms of accuracy, the SVM or logisitic regression classifier? How about in terms of runtime?

**NOTE:** For `sklearns`'s built in classifiers, including svm.SVC, there is a function called `GridSearchCV` which can automatically perform hyperparamater tuning for you. The main advantage of the method (as opposed to writing for-loops) is that it supports parallelization, so it can fit with different parameters at the same time. The function also supports automatic $k$-fold cross-validation (instead of simple train/test split).

You might be interested in using this function in the future. If so, please check out the tutorial in the following lab from previous year: https://github.com/sdrangan/introml/blob/master/unit08_svm/lab_emnist_partial.ipynb.